## リンク先を丸ごとダウンロード
- リンク先のaタグを丸ごとダウンロード

## 相対パスから絶対パスに変換

In [1]:
from urllib.parse import urljoin

base = "http://example.com/html/a.html"

print( urljoin(base, "b.html") )
print( urljoin(base, "sub/c.html") )
print( urljoin(base, "../index.html") )
print( urljoin(base, "../img/hoge.png") )
print( urljoin(base, "../css/hoge.css") )

http://example.com/html/b.html
http://example.com/html/sub/c.html
http://example.com/index.html
http://example.com/img/hoge.png
http://example.com/css/hoge.css


## 再帰的にHTMLページを処理
1. HTMLを解析する。
2. リンクを抽出。
3. 各リンク先について以下の処理を行う。
4. ファイルをダウンロード。
5. ファイルがHTMLならば、再帰的に一からの手順を実行。

## 丸ごとダウンロード　

In [2]:
# Pythonのマニュアルを再帰的にダウンロード 
# モジュールの取り込み --- (※1)
from bs4 import BeautifulSoup
from urllib.request import *   # インターネット上のデータを取得
from urllib.parse import *   # URLの解決を行う
from os import makedirs   # ディレクトリを作成
import os.path, time, re

# 処理済み判断変数 --- (※2)
proc_files = {}   # HTMLファイルの解析を行なったかどうかを判断するための変数を初期化。同じHTMLは二度解析しないようにする。（無限ループ対策）

# HTML内にあるリンクを抽出する関数 --- (※3)
def enum_links(html, base):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("link[rel='stylesheet']") # CSS
    links += soup.select("a[href]") # リンク
    result = []
    # href属性を取り出し、リンクを絶対パスに変換 --- (※4)
    for a in links:
        href = a.attrs['href']
        url = urljoin(base, href)
        result.append(url)
    return result

# ファイルをダウンロードし保存する関数 --- (※5)
def download_file(url):
    o = urlparse(url)
    savepath = "./" + o.netloc + o.path
    if re.search(r"/$", savepath): # ディレクトリならindex.html
        savepath += "index.html"
    savedir = os.path.dirname(savepath)
    # 既にダウンロード済み?
    if os.path.exists(savepath): return savepath
    # ダウンロード先のディレクトリを作成
    if not os.path.exists(savedir):
        print("mkdir=", savedir)
        makedirs(savedir)
    # ファイルをダウンロード --- (※6)
    try:
        print("download=", url)
        urlretrieve(url, savepath)
        time.sleep(1) # 礼儀として1秒スリープ --- (※7)   # Webサーバーに負荷をかけないように。
        return savepath
    except:
        print("ダウンロード失敗:", url)
        return None        

# HTMLを解析してダウンロードする関数 --- (※8)
def analize_html(url, root_url):
    savepath = download_file(url)
    if savepath is None: return
    if savepath in proc_files: return # 解析済みなら処理しない --- (※9)
    proc_files[savepath] = True
    print("analize_html=", url)
    # リンクを抽出 --- (※10)
    html = open(savepath, "r", encoding="utf-8").read()
    links = enum_links(html, url)
    for link_url in links:
        # リンクがルート以外のパスを指していたら無視 --- (※11)
        if link_url.find(root_url) != 0:
            if not re.search(r".css$", link_url): continue
        # HTMLか？
        if re.search(r".(html|htm)$", link_url):
            # 再帰的にHTMLファイルを解析
            analize_html(link_url, root_url)
            continue
        # それ以外のファイル
        download_file(link_url)

if __name__ == "__main__":
    # URLを丸ごとダウンロード --- (※13)
    url = "https://docs.python.jp/3.6/library/"   # サイトを指定。
    analize_html(url, url)

analize_html= https://docs.python.jp/3.6/library/
analize_html= https://docs.python.jp/3.6/library/intro.html
analize_html= https://docs.python.jp/3.6/library/functions.html
analize_html= https://docs.python.jp/3.6/library/constants.html
analize_html= https://docs.python.jp/3.6/library/stdtypes.html
analize_html= https://docs.python.jp/3.6/library/exceptions.html
analize_html= https://docs.python.jp/3.6/library/text.html
analize_html= https://docs.python.jp/3.6/library/string.html
analize_html= https://docs.python.jp/3.6/library/re.html
analize_html= https://docs.python.jp/3.6/library/difflib.html
analize_html= https://docs.python.jp/3.6/library/textwrap.html
analize_html= https://docs.python.jp/3.6/library/unicodedata.html
analize_html= https://docs.python.jp/3.6/library/stringprep.html
analize_html= https://docs.python.jp/3.6/library/readline.html
analize_html= https://docs.python.jp/3.6/library/rlcompleter.html
analize_html= https://docs.python.jp/3.6/library/binary.html
analize_htm

analize_html= https://docs.python.jp/3.6/library/crypto.html
analize_html= https://docs.python.jp/3.6/library/hashlib.html
analize_html= https://docs.python.jp/3.6/library/hmac.html
analize_html= https://docs.python.jp/3.6/library/secrets.html
analize_html= https://docs.python.jp/3.6/library/allos.html
analize_html= https://docs.python.jp/3.6/library/os.html
analize_html= https://docs.python.jp/3.6/library/io.html
analize_html= https://docs.python.jp/3.6/library/time.html
analize_html= https://docs.python.jp/3.6/library/argparse.html
analize_html= https://docs.python.jp/3.6/library/getopt.html
analize_html= https://docs.python.jp/3.6/library/logging.html
analize_html= https://docs.python.jp/3.6/library/logging.config.html
analize_html= https://docs.python.jp/3.6/library/logging.handlers.html
analize_html= https://docs.python.jp/3.6/library/getpass.html
analize_html= https://docs.python.jp/3.6/library/curses.html
analize_html= https://docs.python.jp/3.6/library/curses.ascii.html
analize

analize_html= https://docs.python.jp/3.6/library/frameworks.html
analize_html= https://docs.python.jp/3.6/library/turtle.html
analize_html= https://docs.python.jp/3.6/library/cmd.html
analize_html= https://docs.python.jp/3.6/library/shlex.html
analize_html= https://docs.python.jp/3.6/library/tk.html
analize_html= https://docs.python.jp/3.6/library/tkinter.html
analize_html= https://docs.python.jp/3.6/library/tkinter.ttk.html
analize_html= https://docs.python.jp/3.6/library/tkinter.tix.html
analize_html= https://docs.python.jp/3.6/library/tkinter.scrolledtext.html
analize_html= https://docs.python.jp/3.6/library/idle.html
analize_html= https://docs.python.jp/3.6/library/othergui.html
analize_html= https://docs.python.jp/3.6/library/development.html
analize_html= https://docs.python.jp/3.6/library/typing.html
analize_html= https://docs.python.jp/3.6/library/pydoc.html
analize_html= https://docs.python.jp/3.6/library/doctest.html
analize_html= https://docs.python.jp/3.6/library/unittest.h

analize_html= https://docs.python.jp/3.6/library/grp.html
analize_html= https://docs.python.jp/3.6/library/crypt.html
analize_html= https://docs.python.jp/3.6/library/termios.html
analize_html= https://docs.python.jp/3.6/library/tty.html
analize_html= https://docs.python.jp/3.6/library/pty.html
analize_html= https://docs.python.jp/3.6/library/fcntl.html
analize_html= https://docs.python.jp/3.6/library/pipes.html
analize_html= https://docs.python.jp/3.6/library/resource.html
analize_html= https://docs.python.jp/3.6/library/nis.html
analize_html= https://docs.python.jp/3.6/library/syslog.html
analize_html= https://docs.python.jp/3.6/library/superseded.html
analize_html= https://docs.python.jp/3.6/library/optparse.html
analize_html= https://docs.python.jp/3.6/library/imp.html
analize_html= https://docs.python.jp/3.6/library/undoc.html


In [3]:
# ライブラリの取り込み --- (※1)
from bs4 import BeautifulSoup

# 解析対象となるHTMLを読み込む --- (※2)
html = open("html/eki-link.html", encoding="utf-8").read()

# HTMLを解析する --- (※3)
soup = BeautifulSoup(html, "html.parser")

# <a>タグを抽出する --- (※4)
links = soup.select("a[href]")

# (タイトル, URL)のリストを作る --- (※5)
result = []
for a in links:
    href = a.attrs["href"]
    title = a.string
    result.append((title, href))

print(result)

[('あ', '%E3%81%82'), ('い', '%E3%81%84'), ('う-え', '%E3%81%86-%E3%81%88'), ('お', '%E3%81%8A'), ('か', '%E3%81%8B'), ('き', '%E3%81%8D'), ('く-け', '%E3%81%8F-%E3%81%91'), ('こ', '%E3%81%93'), ('さ', '%E3%81%95'), ('し-しも', '%E3%81%97-%E3%81%97%E3%82%82'), ('しや-しん', '%E3%81%97%E3%82%84-%E3%81%97%E3%82%93'), ('す-そ', '%E3%81%99-%E3%81%9D'), ('た', '%E3%81%9F'), ('ち-て', '%E3%81%A1-%E3%81%A6'), ('と', '%E3%81%A8'), ('な', '%E3%81%AA'), ('に', '%E3%81%AB'), ('ぬ-の', '%E3%81%AC-%E3%81%AE'), ('は', '%E3%81%AF'), ('ひ', '%E3%81%B2'), ('ふ-ほ', '%E3%81%B5-%E3%81%BB'), ('ま', '%E3%81%BE'), ('み', '%E3%81%BF'), ('む-も', '%E3%82%80-%E3%82%82'), ('や-わ行', '%E3%82%84-%E3%82%8F%E8%A1%8C')]


## 連続でダウンロード

In [4]:
from urllib.parse import *
from urllib.request import *
from bs4 import BeautifulSoup
import os, os.path, time

# リンクを抽出する --- (※1)
html = open("html/eki-link.html", encoding="utf-8").read()
soup = BeautifulSoup(html, "html.parser")
links = soup.select("a[href]")
result = []
for a in links:
    href = a.attrs["href"]
    title = a.string
    result.append((title, href))

# リンク先をダウンロードする --- (※2)
savepath = "./out";
if not os.path.exists(savepath): os.mkdir(savepath)
for title, url in result:
    path = savepath + "/" + url + ".html"
    # 相対URLを絶対URLに変換
    a_url = urljoin("http://example.com", url)
    print("download=" + a_url)
    # ここでダウンロードを行う
    # urlretrieve(a_url, path)
    time.sleep(1)

download=http://example.com/%E3%81%82
download=http://example.com/%E3%81%84
download=http://example.com/%E3%81%86-%E3%81%88
download=http://example.com/%E3%81%8A
download=http://example.com/%E3%81%8B
download=http://example.com/%E3%81%8D
download=http://example.com/%E3%81%8F-%E3%81%91
download=http://example.com/%E3%81%93
download=http://example.com/%E3%81%95
download=http://example.com/%E3%81%97-%E3%81%97%E3%82%82
download=http://example.com/%E3%81%97%E3%82%84-%E3%81%97%E3%82%93
download=http://example.com/%E3%81%99-%E3%81%9D
download=http://example.com/%E3%81%9F
download=http://example.com/%E3%81%A1-%E3%81%A6
download=http://example.com/%E3%81%A8
download=http://example.com/%E3%81%AA
download=http://example.com/%E3%81%AB
download=http://example.com/%E3%81%AC-%E3%81%AE
download=http://example.com/%E3%81%AF
download=http://example.com/%E3%81%B2
download=http://example.com/%E3%81%B5-%E3%81%BB
download=http://example.com/%E3%81%BE
download=http://example.com/%E3%81%BF
download=http://exa

## tableタグを解析してCSVファイルとして出力

In [5]:
# BeautifulSoupを利用してHTMLを解析 --- (※1)
from bs4 import BeautifulSoup
html = open("html/eki-link.html", encoding="utf-8").read()
soup = BeautifulSoup(html, "html.parser")

# テーブルを解析する --- (※2)
result = []

# <table>タグを得る --- (※3)
table = soup.select_one("table")

# <tr>タグを得る --- (※4)
tr_list = table.find_all("tr")
for tr in tr_list:
    # <td>あるいは<th>タグを得る --- (※5)
    result_row = []
    td_list = tr.find_all(["td","th"])
    for td in td_list:
        cell = td.get_text()
        result_row.append(cell)
    result.append(result_row)

# リストをCSVファイルとして出力 --- (※6)
for row in result:
    print(",".join(row))

あ,い,う-え,お,か,き,く-け
こ,さ,し-しも,しや-しん,す-そ,た,ち-て
と,な,に,ぬ-の,は,ひ,
ふ-ほ,ま,み,む-も,や-わ行,,


## SNSで特定のユーザーの作品を全部お気に入りにする

In [6]:
from selenium.webdriver import Firefox, FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

USER = "JS-TESTER"
PASS = "ipCU12ySxI"
FAV_USER_ID = 32 # お気に入りをつけるユーザーのID
SNS_URL = "https://uta.pw/sakusibbs/"

# Firefoxのドライバを得る --- (※1)
options = FirefoxOptions()
options.add_argument('-headless')
browser = Firefox(options=options)

# ログインする --- (※2)
url_login = SNS_URL + "users.php?action=login"
browser.get(url_login)

# テキストボックスに文字を入力してフォーム送信する関数
def form_post(frm, d):
    for field, value in d.items():
        e = frm.find_element_by_name(field)
        e.clear()
        e.send_keys(value)
    frm.submit()
    # ページのロード完了まで待機
    WebDriverWait(browser, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, ".islogin")))

# 対象となるフォームを指定
frm = browser.find_element_by_css_selector("#loginForm form")
# テキストボックスにデータを指定して送信する
form_post(frm, {
    "username_mmlbbs6":USER, 
    "password_mmlbbs6":PASS} )

# 本当にログインしたか画像で確認してみる --- (※3)
browser.save_screenshot("sns-logined.png")
# 本当にログインしたかHTMLで判断してみる --- (※4)
e = browser.find_element_by_id("bbsheader")
html = e.get_attribute("innerHTML")
if html.find("action=logout") < 0:
    print("ログインできていません(ToT)")
    quit()
print("+ ログインしました")
time.sleep(1)

# ユーザーのページ(作品一覧)を開く --- (※5)
url = SNS_URL + "users.php?user_id=" + str(FAV_USER_ID)
browser.get(url)

# 作品一覧を得る --- (※6)
sakuhin_list = []
links = browser.find_elements_by_css_selector(
    "ul#mmlist li a")
for a in links:
    href = a.get_attribute('href')
    title = a.text
    sakuhin_list.append((href, title))
print("+ 作品の一覧を{0}件取得しました".format(len(sakuhin_list)))

# 一気にお気に入りを付ける --- (※7)
for href, title in sakuhin_list:
    # 作品ページを開く
    print("- ", title)
    browser.get(href)
    try:
        # お気に入りボタンを得る --- (※8)
        e = browser.find_element_by_id("fav_add_btn")
        e.click()
        # お気に入りを取り消す場合 --- (※9)
        # e = browser.find_element_by_id("fav_remove_btn")
        # e.click()
        print("| お気に入りにしました!")
    except:
        # お気に入りボタンがなかった時
        print("| 既にお気に入りでした.")
    # 負荷軽減
    time.sleep(1)

+ ログインしました
+ 作品の一覧を9件取得しました
-  Learning Python Song
| 既にお気に入りでした.
-  Learning Python
| 既にお気に入りでした.
-  ぱいソング
| 既にお気に入りでした.
-  テスト
| 既にお気に入りでした.
-  吾輩はテスターである　その２
| 既にお気に入りでした.
-  吾輩はテスターである
| 既にお気に入りでした.
-  プリンとシュークリーム
| 既にお気に入りでした.
-  眠ってもデバッグ
| 既にお気に入りでした.
-  今日も明日もJS三昧
| 既にお気に入りでした.
